In [1]:
import os
os.chdir("../")

In [2]:
from pprint import pprint

In [3]:
import torch

model = torch.load("artifacts/training/model.pt", map_location="mps", weights_only=False)

In [4]:
# Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [5]:
# Configuration manager
from chestCancerClassifier.constants import *
from chestCancerClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.pt",
            training_data="artifacts/data_ingestion/Chest-CT-Scan",
            mlflow_uri="https://dagshub.com/sazuyakun/chest-cancer-classifier.mlflow/",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [14]:
# Components

import mlflow.pytorch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import mlflow
import mlflow.pytorch
from pathlib import Path
from urllib.parse import urlparse
import json


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

        if torch.backends.mps.is_available():
            self.device = torch.device("mps")
        elif torch.cuda.is_available():
            self.device = torch.device("cuda")
        else:
            self.device = torch.device("cpu")

    def _valid_loader(self):
        image_size = self.config.params_image_size[:-1]  # (H, W)
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]

        valid_transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])

        full_dataset = datasets.ImageFolder(root=self.config.training_data)
        val_size = int(0.3 * len(full_dataset))
        train_size = len(full_dataset) - val_size

        _, self.valid_dataset = random_split(full_dataset, [train_size, val_size])
        self.valid_dataset.dataset.transform = valid_transform

        self.valid_loader = DataLoader(self.valid_dataset, batch_size=self.config.params_batch_size, shuffle=False)

    @staticmethod
    def load_model(path: Path):
        model = torch.load(path, map_location=torch.device("cpu"), weights_only=False)
        return model

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.model.to(self.device)
        self.model.eval()

        self._valid_loader()
        criterion = nn.CrossEntropyLoss()

        total_loss = 0.0
        total_correct = 0
        total_samples = 0

        with torch.no_grad():
            for inputs, labels in tqdm(self.valid_loader):
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)

                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                total_loss += loss.item() * inputs.size(0)
                total_correct += torch.sum(preds == labels).item()
                total_samples += inputs.size(0)

        avg_loss = total_loss / total_samples
        accuracy = total_correct / total_samples
        self.score = (avg_loss, accuracy)
        print(f"Validation Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        with open("scores.json", "w") as f:
            json.dump(scores, f, indent=4)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(self.config.mlflow_uri).scheme

        with mlflow.start_run():
            mlflow.log_param(
                key="params",
                value=self.config.all_params
            )
            mlflow.log_metrics(
                {
                    "loss": self.score[0],
                    "accuracy": self.score[1]
                }
            )

            mlflow.pytorch.autolog()

In [15]:
import dagshub
dagshub.init(repo_owner='sazuyakun', repo_name='chest-cancer-classifier', mlflow=True)

try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-06-25 03:01:06,442: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/sazuyakun/chest-cancer-classifier "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "sazuyakun/chest-cancer-classifier"

[2025-06-25 03:01:06,445: INFO: helpers: Initialized MLflow to track repo "sazuyakun/chest-cancer-classifier"]


Repository sazuyakun/chest-cancer-classifier initialized!

[2025-06-25 03:01:06,446: INFO: helpers: Repository sazuyakun/chest-cancer-classifier initialized!]
[2025-06-25 03:01:06,447: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-25 03:01:06,448: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-25 03:01:06,449: INFO: common: created directory at: artifacts]


100%|██████████| 7/7 [00:02<00:00,  3.01it/s]


Validation Loss: 0.3332, Accuracy: 0.9804
🏃 View run adaptable-ape-272 at: https://dagshub.com/sazuyakun/chest-cancer-classifier.mlflow/#/experiments/0/runs/c58fbd2793434443a41c6a9ed6e2c642
🧪 View experiment at: https://dagshub.com/sazuyakun/chest-cancer-classifier.mlflow/#/experiments/0
